# eICU-CRD query API tutorial

This notebook shows examples of how to use the cyclops.query API on [eicu-CRD](https://eicu-crd.mit.edu/).

Each query is limit to 100 rows (for quick results).

* First, setup the eICU database according to the instructions in [eicu-code](https://github.com/MIT-LCP/eicu-code/tree/master/build-db/postgres).
* The database is assumed to be hosted using postgres. Update the config parameters such as username and password, passed to `EICUQuerier` accordingly.

## Imports and instantiate `EICUQuerier`

In [ ]:
import cyclops.query.ops as qo
from cyclops.query import EICUQuerier

querier = EICUQuerier(
    dbms="postgresql",
    port=5432,
    host="localhost",
    database="eicu",
    user="postgres",
    password="pwd",
)
# List all tables.
querier.list_tables()

## Example 1. Get all female patients discharged in 2014 (limit to 100 rows).

In [ ]:
ops = qo.Sequential(
    [
        qo.ConditionEquals("hospitaldischargeyear", 2014),
        qo.ConditionEquals("gender", "Female"),
    ]
)
patients = querier.eicu_crd.patient(ops=ops).run(limit=100)
print(f"{len(patients)} rows extracted!")

## Example 2. Get all patient encounters with diagnoses (`schizophrenia` in `diagnosisstring`), discharged in the year 2015.

In [ ]:
ops = qo.Sequential(
    [
        qo.ConditionEquals("hospitaldischargeyear", 2015),
    ]
)
patients = querier.eicu_crd.patient(ops=ops)
diagnosis_ops = qo.Sequential(
    [
        qo.ConditionSubstring("diagnosisstring", "schizophrenia"),
    ]
)
diagnoses = querier.eicu_crd.diagnosis(
    join=qo.JoinArgs(
        join_table=patients.query,
        on="patientunitstayid",
    ),
    ops=diagnosis_ops,
).run(limit=100)
print(f"{len(diagnoses)} rows extracted!")

## Example 3. Get potassium lab tests for patients discharged in the year 2014, for all teaching hospitals.

In [ ]:
hospitals = querier.eicu_crd.hospital()
ops = qo.Sequential(
    [
        qo.ConditionEquals("hospitaldischargeyear", 2015),
        qo.ConditionEquals("teachingstatus", True),
    ]
)
patients = querier.eicu_crd.patient(
    join=qo.JoinArgs(join_table=hospitals.query, on="hospitalid"), ops=ops
)
lab_ops = qo.Sequential(
    [
        qo.ConditionEquals("labname", "potassium"),
    ]
)
labs = querier.eicu_crd.lab(
    join=qo.JoinArgs(
        join_table=patients.query,
        on="patientunitstayid",
    ),
    ops=lab_ops,
).run(limit=100)
print(f"{len(labs)} rows extracted!")

## Example 4. Get glucose medications (substring search) for female patients discharged in 2014.

In [ ]:
ops = qo.Sequential(
    [
        qo.ConditionEquals("hospitaldischargeyear", 2014),
        qo.ConditionEquals("gender", "Female"),
    ]
)
patients = querier.eicu_crd.patient(ops=ops)
medications_ops = qo.Sequential(
    [
        qo.ConditionSubstring("drugname", "glucose"),
    ]
)
medications = querier.eicu_crd.medication(
    join=qo.JoinArgs(join_table=patients.query, on="patientunitstayid"),
    ops=medications_ops,
).run(limit=100)
print(f"{len(medications)} rows extracted!")